<a href="https://colab.research.google.com/github/Soy-code/one-day-LLM-FT/blob/main/Training_a_causal_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a causal language model

라이브러리 설치

In [2]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


### 데이터셋 가져오기

Python 코드는 GitHub과 같은 코드 저장소에서 풍부하게 제공되며, 모든 Python 저장소를 스크래핑하여 데이터셋을 생성하는 데 사용할 수 있습니다. 약 180GB에 달하는 GitHub 덤프를 사용하여 'codeparrot'이라는 이름으로 약 2천만 개의 Python 파일을 포함하는 데이터셋을 구축하고, 이를 Hugging Face Hub에 공유했습니다.

따라서 transformersbook/codeparrot 데이터셋은 Hugging Face 데이터셋 허브에 호스팅되어 있는 데이터셋으로, 자연어 처리(NLP) 및 기계 학습 분야에서 사용할 수 있는 프로그래밍 코드 샘플을 포함하고 있습니다. 이 데이터셋은 프로그래밍 언어를 이해하고 생성할 수 있는 AI 모델을 개발하는 데 중점을 둡니다.

그러나 전체 코퍼스에서 훈련하는 것은 시간과 계산 리소스를 많이 소모하며, 우리는 필요한 데이터셋의 부분집합을 필터링하기 위해 아래와 같은 코드를 작성할 수 있습니다.

In [3]:
def any_keyword_in_string(string, keywords):
    for keyword in keywords:
        if keyword in string:
            return True
    return False

In [4]:
filters = ["pandas", "sklearn", "matplotlib", "seaborn"]   # 이것에 관련한 데이터들만 필터링해서 사용하겠다
example_1 = "import numpy as np"
example_2 = "import pandas as pd"

print(
    any_keyword_in_string(example_1, filters), any_keyword_in_string(example_2, filters)
)

False True


In [5]:
from collections import defaultdict
from tqdm import tqdm
from datasets import Dataset


def filter_streaming_dataset(dataset, filters):
    filtered_dict = defaultdict(list)
    total = 0
    for sample in tqdm(iter(dataset)):
        total += 1
        if any_keyword_in_string(sample["content"], filters):
            for k, v in sample.items():
                filtered_dict[k].append(v)
    print(f"{len(filtered_dict['content'])/total:.2%} of data after filtering.")
    return Dataset.from_dict(filtered_dict)

위 함수는 데이터셋에서 키워드(pandas, sklearn, matplotlib, seaborn)와 연관된 데이터를 필터링합니다. 그러나 매우 시간소모가 크기 때문에 직접 수행하지 않겠습니다.

In [6]:
# This cell will take a very long time to execute, so you should skip it and go to
# the next one!
# from datasets import load_dataset

# split = "train"  # "valid"
# filters = ["pandas", "sklearn", "matplotlib", "seaborn"]

# data = load_dataset(f"transformersbook/codeparrot-{split}", split=split, streaming=True)
# filtered_data = filter_streaming_dataset(data, filters)

아래 코드는 위에서 우리가 살펴본 필터링 이후 전체 데이터셋의 3.26%에 해당하는 8.25G의 텍스트 데이터 다운로드합니다. 몇 분을 소요합니다.

In [7]:
from datasets import load_dataset, DatasetDict

ds_train = load_dataset("huggingface-course/codeparrot-ds-train", split="train")
ds_valid = load_dataset("huggingface-course/codeparrot-ds-valid", split="validation")

raw_datasets = DatasetDict(
    {
        "train": ds_train.shuffle().select(range(500)),  # 500개씩만 sampling
        "valid": ds_valid.shuffle().select(range(500))
    }
)

# DatasetDict({
#     train: Dataset({
#         features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
#         num_rows: 606720
#     })
#     valid: Dataset({
#         features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
#         num_rows: 3322
#     })
# })


raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


codeparrot-ds-train.jsonl:   0%|          | 0.00/8.25G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

codeparrot-ds-valid.jsonl:   0%|          | 0.00/46.1M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/3322 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 500
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 500
    })
})

이 데이터셋에서 예제를 살펴보겠습니다. 각 필드의 처음 200자만 보여드리겠습니다:

In [8]:
for key in raw_datasets["train"][5]:
    print(f"{key.upper()}: {raw_datasets['train'][0][key][:300]}")

REPO_NAME: Rossonero/bmlswp
PATH: ch07/boston1.py
COPIES: 22
SIZE: 1147
CONTENT: # This code is supporting material for the book
# Building Machine Learning Systems with Python
# by Willi Richert and Luis Pedro Coelho
# published by PACKT Publishing
#
# It is made available under the MIT License

# This script shows an example of simple (ordinary) linear regression

# The first 
LICENSE: mit


In [9]:
# 데이터 출력을 깔끔하게 보여줍니다. (탭과 줄바꿈 적용)
for i, content in enumerate(raw_datasets["train"][:2]["content"]):
    print(f"Content {i + 1}:")
    print(content)
    print()

Content 1:
# This code is supporting material for the book
# Building Machine Learning Systems with Python
# by Willi Richert and Luis Pedro Coelho
# published by PACKT Publishing
#
# It is made available under the MIT License

# This script shows an example of simple (ordinary) linear regression

# The first edition of the book NumPy functions only for this operation. See
# the file boston1numpy.py for that version.

import numpy as np
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt

boston = load_boston()
x = boston.data
y = boston.target

# Fitting a model is trivial: call the ``fit`` method in LinearRegression:
lr = LinearRegression()
lr.fit(x, y)

# The instance member `residues_` contains the sum of the squared residues
rmse = np.sqrt(lr.residues_/len(x))
print('RMSE: {}'.format(rmse))

fig, ax = plt.subplots()
# Plot a diagonal (for reference):
ax.plot([0, 50], [0, 50], '-', color=(.9,.3,.3), lw=4)



먼저 데이터를 토큰화하여 훈련에 사용할 수 있도록 합니다.

우리의 목표는 주로 짧은 함수 호출을 자동 완성하는 것으로 컨텍스트 크기를 상대적으로 작게 유지할 것 입니다.이로 인해 모델을 훨씬 빠르게 훈련시킬 수 있고, 훨씬 적은 메모리가 필요합니다. 만약 응용 프로그램에서 더 많은 컨텍스트가 중요하다면(예를 들어, 함수 정의가 포함된 파일을 바탕으로 단위 테스트를 작성하게 하고 싶은 경우), 그 숫자를 증가시키는 것이 좋습니다.

하지만 이것이 더 큰 GPU 메모리 사용량을 수반합니다. 일단 지금은 GPT-2나 GPT-3에서 사용된 1,024 또는 2,048 토큰 대신 128 토큰으로 컨텍스트 크기를 고정하겠습니다.

대부분의 문서는 128 토큰보다 많으므로, 입력을 최대 길이로 단순히 자르는 것은 데이터셋의 큰 부분을 제거하게 됩니다. 이때 return_overflowing_tokens 옵션을 사용하면 전체 입력을 토큰화하면서 동시에 여러 청크로 나눌 수 있습니다. 또한 return_length 옵션을 사용하여 생성된 각 청크의 길이를 자동으로 반환하겠습니다. 대부분의 경우 마지막 청크는 컨텍스트 크기보다 작을 것이고, 우리는 이러한 부분을 제거할 것입니다; 패딩 문제를 피하기 위해서이며, 어차피 충분한 데이터가 있기 때문에 이 부분이 필요하지 않습니다.

In [10]:
from transformers import AutoTokenizer

context_length = 128
#context_length는 생성할 토큰의 최대 길이를 설정

tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")
#"huggingface-course/code-search-net-tokenizer" 모델을 로드합니다. 이 토크나이저는 코드 검색과 관련된 작업에 최적화된 특별한 설정과 토큰화 규칙을 갖고 있습니다.

outputs = tokenizer(
    raw_datasets["train"][:2]["content"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

# raw_datasets["train"][:2]["content"]는 훈련 데이터셋에서 처음 두 개의 샘플을 가져옵니다. 이 데이터는 'content' 키로 접근할 수 있습니다.
# truncation=True는 토큰 길이가 max_length를 초과할 경우 뒷부분을 잘라내도록 설정합니다.
# return_overflowing_tokens=True는 최대 길이를 초과하는 토큰들을 별도로 반환하도록 합니다. 이는 데이터가 max_length보다 길 경우 여러 부분으로 나뉘어 처리됨을 의미합니다.
# return_length=True는 각 토큰화된 입력의 길이를 반환하도록 설정합니다.

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

#Chunk Mapping: 각 청크가 원본 데이터셋의 어느 샘플에서 왔는지를 나타냅니다. 배열의 값 0은 첫 번째 샘플에서 나온 청크들임을, 1은 두 번째 샘플에서 나온 청크들임을 나타냅니다.

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Input IDs length: 10
Input chunk lengths: [128, 128, 86, 128, 128, 128, 128, 128, 128, 43]
Chunk mapping: [0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [11]:
tokenizer.vocab

{'PULL': 43103,
 'rmq': 42028,
 'ĠFlux': 39162,
 'DT': 11595,
 'ĠIte': 9515,
 'AsString': 47782,
 'Bitmap': 31501,
 'hemer': 14154,
 'pherd': 31803,
 'decoded': 12987,
 'åĻ¨': 23748,
 'Unexpected': 8926,
 "Ġ'')))": 42023,
 'ĠAcquire': 41198,
 'stddevs': 17964,
 'scenarios': 31723,
 'ĠRefresh': 20988,
 'Ġcook': 22281,
 'Ġerrstr': 45311,
 'ĠSynchron': 40017,
 'Ġspacy': 36823,
 'Ġ___': 47615,
 'Ġcollected': 10540,
 'Ġretrieves': 21171,
 'rdist': 44407,
 'Ġfine': 11626,
 'Ġgtf': 25711,
 'Ġtic': 8353,
 'isVisible': 29849,
 'lets': 8029,
 'Ġtend': 21005,
 'Conversation': 44189,
 'Ġph': 1912,
 'Met': 4714,
 'Disposition': 16985,
 'ĠSASL': 47879,
 'Ġvals': 5285,
 '________________': 13559,
 'imer': 3891,
 'lium': 28910,
 'ĠToken': 7530,
 'harvest': 32857,
 'specifications': 36850,
 '}``.': 42490,
 'Phenotype': 43673,
 'ĠThrow': 34120,
 'çķ': 27978,
 'triu': 30379,
 'åĲĪ': 21131,
 'ToContents': 39323,
 'Ġhalf': 7504,
 'wcs': 6685,
 'ĠBig': 12275,
 'CAR': 22485,
 'isna': 48537,
 'Ġamazon': 38275

In [12]:
len(tokenizer.vocab)

50000

이제 전체 데이터셋을 tokenize 함수로 전처리합니다.

In [13]:
def tokenize(element):
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length: #짧은 길이의 마지막 chunk는 제거될 것 입니다.
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["valid"].column_names
)
tokenized_datasets

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 13646
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 14136
    })
})

### 모델 초기화하기

우리는 GPT-2 모델을 새롭게 초기화할 것입니다. 우리의 모델에 대해서는 작은 GPT-2 모델과 동일한 구성을 사용할 것이므로, 사전 훈련된 구성을 불러오고 토크나이저의 크기가 모델 어휘 사전 크기와 일치하는지 확인한 후, 시퀀스의 시작과 끝을 나타내는 bos (beginning of sequence) 및 eos (end of sequence) 토큰 ID를 전달할 것입니다.

In [14]:
tokenizer.vocab

{'PULL': 43103,
 'rmq': 42028,
 'ĠFlux': 39162,
 'DT': 11595,
 'ĠIte': 9515,
 'AsString': 47782,
 'Bitmap': 31501,
 'hemer': 14154,
 'pherd': 31803,
 'decoded': 12987,
 'åĻ¨': 23748,
 'Unexpected': 8926,
 "Ġ'')))": 42023,
 'ĠAcquire': 41198,
 'stddevs': 17964,
 'scenarios': 31723,
 'ĠRefresh': 20988,
 'Ġcook': 22281,
 'Ġerrstr': 45311,
 'ĠSynchron': 40017,
 'Ġspacy': 36823,
 'Ġ___': 47615,
 'Ġcollected': 10540,
 'Ġretrieves': 21171,
 'rdist': 44407,
 'Ġfine': 11626,
 'Ġgtf': 25711,
 'Ġtic': 8353,
 'isVisible': 29849,
 'lets': 8029,
 'Ġtend': 21005,
 'Conversation': 44189,
 'Ġph': 1912,
 'Met': 4714,
 'Disposition': 16985,
 'ĠSASL': 47879,
 'Ġvals': 5285,
 '________________': 13559,
 'imer': 3891,
 'lium': 28910,
 'ĠToken': 7530,
 'harvest': 32857,
 'specifications': 36850,
 '}``.': 42490,
 'Phenotype': 43673,
 'ĠThrow': 34120,
 'çķ': 27978,
 'triu': 30379,
 'åĲĪ': 21131,
 'ToContents': 39323,
 'Ġhalf': 7504,
 'wcs': 6685,
 'ĠBig': 12275,
 'CAR': 22485,
 'isna': 48537,
 'Ġamazon': 38275

In [15]:
len(tokenizer.vocab)

50000

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [17]:
# 모델 생성 (config를 사용)
model = AutoModelForCausalLM.from_config(config)


model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.2M parameters


훈련을 시작하기 전에, 배치를 생성할 데이터 콜레이터를 설정해야 합니다. DataCollatorForLanguageModeling는 언어 모델링을 위해 특별히 설계되었습니다. 배치를 쌓고 패딩하는 것 외에도, 이 데이터 콜레이터는 언어 모델 레이블을 생성하는 작업도 처리합니다. DataCollatorForLanguageModeling은 마스크 언어 모델링(MLM)과 인과적 언어 모델링(CLM)을 모두 지원합니다. 기본적으로는 MLM을 위한 데이터를 준비하지만, mlm=False를 설정함으로써 CLM으로 전환할 수 있습니다:

In [18]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)  # mlml = False로 두면, labels이 자동으로 생성됨

아래 코드를 실행해서 각 샘플이 같은 shape의 tensor로 쌓인 것을 확인하세요.

In [19]:
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

# labels : 기존에 labels가 없었는데 생김


input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


레이블을 입력과 정렬하기 위해 한 칸 씩 이동시키는 작업은 모델 내부에서 일어나므로, 데이터 콜레이터는 입력을 복사하여 레이블을 생성하기만 합니다.

In [20]:
out['input_ids'][0][:10]

tensor([    3,   841,  1046,   300, 18870, 10238,   296,   256,  7303,   173])

In [21]:
out['labels'][0][:10]

tensor([    3,   841,  1046,   300, 18870, 10238,   296,   256,  7303,   173])

[TrainingArguments](https://huggingface.co/docs/transformers/v4.39.3/en/main_classes/trainer#transformers.TrainingArguments)는 모델 학습에 대한 다양한 옵션을 설정할 수 있습니다. 배치 사이즈는 256( per_device_train_batch_size*gradient_accumulation_steps)을 사용해줍니다.

In [23]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="codeparrot-ds",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=10, #5_000
    logging_steps=10, #5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=5,
    weight_decay=0.1,
    warmup_steps=10, #1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=False,
    report_to=['tensorboard'],
)

# output_dir="codeparrot-ds": 훈련 중 생성된 모델과 기타 출력 파일들을 저장할 디렉토리 경로입니다.
# per_device_train_batch_size=32: 각 디바이스(예: GPU)에 대한 훈련 배치 크기입니다.
# per_device_eval_batch_size=32: 각 디바이스에 대한 평가 배치 크기입니다.
# evaluation_strategy="steps": 평가를 실행할 전략을 지정합니다. "steps"는 일정 스텝마다 평가를 실행하게 합니다.
# eval_steps=5_000: 평가를 실행할 스텝 간격을 설정합니다.
# logging_steps=5_000: 로깅을 실행할 스텝 간격입니다. 훈련 진행 상황을 기록합니다.
# gradient_accumulation_steps=8: 몇 번의 훈련 스텝 후에 그래디언트를 업데이트할지 설정합니다. 이는 메모리 사용을 최적화하는 데 도움을 줄 수 있습니다.
# num_train_epochs=1: 모델을 전체 데이터셋을 사용해 훈련할 에폭 수입니다.
# weight_decay=0.1: 가중치 감소(Regularization)를 위한 계수입니다. 모델의 과적합을 방지하는 데 도움을 줍니다.
# warmup_steps=1_000: 학습률 스케줄러가 본격적으로 시작되기 전에 진행할 웜업 스텝의 수입니다.
# lr_scheduler_type="cosine": 학습률 스케줄러의 유형을 지정합니다. "cosine"은 학습률을 코사인 함수의 형태로 점차 감소시키는 방식입니다.
# learning_rate=5e-4: 초기 학습률 설정입니다.
# save_steps=5_000: 모델을 저장할 스텝 간격입니다.
# fp16=True: 16비트 부동소수점(Floating Point) 연산을 사용합니다. 이는 GPU에서의 메모리 사용량과 연산 속도를 최적화하는 데 도움을 줍니다.
# push_to_hub=True: 훈련이 완료된 모델을 Hugging Face Hub에 자동으로 푸시합니다. 이를 통해 모델을 공유하고 버전 관리할 수 있습니다. 로그인이 필요하므로 False로 설정합니다.

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)

<ipython-input-24-0a8fcd383b0d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
trainer.train()

Step,Training Loss,Validation Loss
10,9.426600,7.860912
20,7.090800,6.873227
30,6.660100,6.539821
40,6.326200,6.296742
50,6.060700,6.010552
60,5.719200,5.776795
70,5.459100,5.599476
80,5.262400,5.470365
90,5.156000,5.371544
100,5.019300,5.272265


TrainOutput(global_step=265, training_loss=5.0023269293443215, metrics={'train_runtime': 452.6533, 'train_samples_per_second': 150.733, 'train_steps_per_second': 0.585, 'total_flos': 4426809606144000.0, 'train_loss': 5.0023269293443215, 'epoch': 4.964871194379391})

모델을 업로드하기 위해서는 token을 받아와 huggingface hub에 로그인하고, push_to_hub 메서드로 모델을 업로드합니다.

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
#trainer.push_to_hub()

### 학습완료된 모델 사용

In [26]:
import torch
from transformers import pipeline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline(
    "text-generation", model="huggingface-course/codeparrot-ds", max_length=128, device=device
)

config.json:   0%|          | 0.00/938 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [27]:
txt = """\
# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
"""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
plt.scatter(x, y, s=10, c='g')
plt.scatter(x, y, s=5, c='r')
plt.legend()
plt.title('single exponential')
# decorate
plt.xlim((0, 1))
plt.ylim((0.8, 2))
plt.xlabel('value')
plt.ylabel('mean')
plt.show()
plt.


In [28]:
txt = """\
# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create dataframe from x and y
"""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create dataframe from x and y
df = pd.DataFrame(np.random.randn(100))

# create dataframe
df = pd.DataFrame(np.random.randn(100, 10))

# create a list containing x and y
x2 = [[0] * 10, [100] * 10, [400] * 10, [500] * 10, [100] * 10,
         [50] * 10, [100] * 10


In [29]:
txt = """\
# dataframe with profession, income and name
df = pd.DataFrame({'profession': x, 'income':y, 'name': z})

# calculate the mean income per profession
"""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


# dataframe with profession, income and name
df = pd.DataFrame({'profession': x, 'income':y, 'name': z})

# calculate the mean income per profession
mean_income = df.groupby('province').mean()[['province', 'code']].count(drop=True)
mean_income = mean_income[['code', 'code']].mean()

# calculate the mean income vs. profession, income and name
mean_income = df.groupby('province').mean()[['province', 'code']].sum(drop=True)
mean


In [30]:
txt = """
# import random forest regressor from scikit-learn
from sklearn.ensemble import RandomForestRegressor

# fit random forest model with 300 estimators on X, y:
"""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



# import random forest regressor from scikit-learn
from sklearn.ensemble import RandomForestRegressor

# fit random forest model with 300 estimators on X, y:
# (the last call of fit is the final line)
rf.fit(X, y)

# predict the total return value
rf_y = rf.predict(X)  

# Plot the prediction and the predictions
fig, ax = plt.subplots(2, 1, figsize=(12, 10))
ax[0].plot(rf_y[:, 0], rf_y[:, 1])
ax[0].
